In [1]:
import pya # install this package via pip install klayout. NOT via pip install pya. 
import numpy as np
from operator import xor
import matplotlib.bezier as mbz

In [2]:
def merge_polygons(polys):
    '''Merges the shapes in polys into one polygon. Overlaps will be merged into one.
    
    @param list polys: list of the polygons that should be merged.

    @return poly.Region: region containing the merged polygon
    '''
    union = pya.Region()
    for poly in polys:
        union += pya.Region(poly)
    merged = union.merge()
    return merged


def merge_regions(regions):
    '''Merges the regions in regions into one region. Overlaps will be merged into one?
    
    @param list regions: list of the regions that should be merged.

    @return Region: region containing the merged regions
    '''
    union = pya.Region()
    try: # try if a list of regions is passed
        for region in regions:
            union += region
    except: # if only single region is passed
        union.insert(regions)
    merged = union.merge()
    return merged


def move_region(region,offset):
    """Moves the given region by the vector offset.

    @param Region region: region that should be moved

    @param list offset: vector by which the region should be moved.
    """

    _region = region.dup()
    _region.move(offset[0],offset[1])

    return _region


def rotate_region(region,angle):
    """ Rotates the region by multiples of 90° around its center.
    Center refers to the middle between the max and min in x and y.

    @param Region region: region that should be rotated
    
    @param int angle: rotation angle: 1 is 90°, 2 is 180°, 3 is 270°
    """

    boundingbox = region.bbox()
    x = boundingbox.left + boundingbox.width()/2
    y = boundingbox.bottom + boundingbox.height()/2
    rotation = pya.Trans(angle, False, pya.Point(0,0)) # False: no mirroring
    _region = region.dup()
    _region = move_region(_region,[-x,-y])
    _region.transform(rotation)
    _region = move_region(_region,[x,y])

    return _region

def mirror_region(region,axis='y'):
    """ Rotates the region by multiples of 90° around its center.
    Center refers to the middle between the max and min in x and y.

    @param Region region: region that should be rotated
    
    @param int angle: rotation angle: 1 is 90°, 2 is 180°, 3 is 270°
    """

    boundingbox = region.bbox()
    x = boundingbox.left + boundingbox.width()/2
    y = boundingbox.bottom + boundingbox.height()/2
    if axis == 'x':
        t = pya.Trans(2, 'm0', pya.Point(0,0))
    elif axis == 'y':
        t = pya.Trans(0, 'm0', pya.Point(0,0))
    else:
        raise ValueError('Given axis not supported.')
    _region = region.dup()
    _region = move_region(_region,[-x,-y])
    _region.transform(t)
    _region = move_region(_region,[x,y])

    return _region

#######################################
# below here are properly refactored functions

def create_rectangle(corners=None, posi=None, dims=None, anchor=None):
    '''Creates a rectangle (box object) at the given location.
    
    Either give the coordinates of the lower left and upper right corner or a point plus dimensions.

    @param list corners: 4 element list of the corners as follows: [lowerleft_x, lower_left_y, upper_right_x, upper_right_y].

    @param list posi: 2 element list of the reference point as [x,y].

    @param list dims: Width and height of the rectangle as [width, height].

    @param str anchor: Location of the anchor. Supported are 'center', 'sw' and 's'.

    @return box: Box object.
    '''

    # check if all necessary (and not more) parameters are given
    if corners==None and posi!=None and dims!=None and anchor!=None:
        if anchor == 'center':
            lowerleft_x = posi[0]-dims[0]/2
            lower_left_y = posi[1]-dims[1]/2
            upper_right_x = posi[0]+dims[0]/2
            upper_right_y = posi[1]+dims[1]/2
        elif anchor =='sw':
            lowerleft_x = posi[0]
            lower_left_y = posi[1]
            upper_right_x = posi[0]+dims[0]
            upper_right_y = posi[1]+dims[1]
        elif anchor == 's':
            lowerleft_x = posi[0]-dims[0]/2
            lower_left_y = posi[1]
            upper_right_x = posi[0]+dims[0]/2
            upper_right_y = posi[1]+dims[1]
        else: 
            raise Exception('Unknown anchor "/s" used.'%str(anchor))
        corners = [lowerleft_x, lower_left_y, upper_right_x, upper_right_y]
    elif corners!=None and posi==None and dims==None and anchor==None:
        pass
    else:
        raise Exception('Bad combination of input arguments.')
    
    box = pya.Box.new(pya.Point(corners[0],corners[1]),pya.Point(corners[2],corners[3]))
    
    return box


def create_polygon(pts_list):
    '''Creates a polygon out of the given point list.

    @param list pts_list: List of the points.
        give it like
        [(x1,y1),(x2,y2),(x3,y3),...]
        As long as you can index it as [i][j] it is fine.
    
    @return poly: Polygon object
    '''

    structure = []
    for pt in pts_list:
        structure.append(pya.Point.new(pt[0],pt[1]))
    poly = pya.Polygon.new(structure)
    return poly


def create_text(posi,gridsize,height,text,anchor='sw'):
    """Creates a text object at the specified location.
    
    @param list posi: x and y coordinate of the box (lower left corner)

    @param float gridsize: size of the used grid (layout.dbu)

    @param float height: height of the text

    @param str text: ext to display

    @param str anchor: loction of the anchor.
        Supported are 'sw', 'se', 'ne', 'nw' and 'c'.
    
    """
    scale = height * 10/7 * 1e-3 # rescaling so size is in dbu
    region = pya.TextGenerator.default_generator().text(text,gridsize/scale)
    txtheight = region.bbox().height()
    txtwidth = region.bbox().width()
    if anchor == 'sw':
        region.move(int(posi[0]), int(posi[1]))
    elif anchor == 'se':
        region.move(int(posi[0])-txtwidth, int(posi[1]))
    elif anchor == 'ne':
        region.move(int(posi[0])-txtwidth, int(posi[1])-txtheight)
    elif anchor == 'nw':
        region.move(int(posi[0]), int(posi[1])-txtheight)
    elif anchor == 'c':
        region.move(int(posi[0])-txtwidth/2, int(posi[1])-txtheight/2)
    
    else: 
        print('anchor unknown, using bottomleft.')
    return region


def create_marker(posi,width):
    """Creates a marker that is centered at posi and width wide and tall.

    @param list posi: center position of the marker

    @param int width: width of the marker
    """
    boxlength = 0.3*width
    boxwidth = 0.1*width
    connectorwidth = 0.01*width
    arms_offset = 0.5*width-0.5*boxlength

    elements = []
    elements.append(create_rectangle(posi=posi, dims=[width,connectorwidth],anchor='center')) # connector_vertical
    elements.append(create_rectangle(posi=posi, dims=[connectorwidth,width], anchor='center')) # connector_horizontal
    elements.append(create_rectangle(posi=[posi[0],posi[1]+arms_offset],dims=[boxwidth,boxlength],anchor='center')) # upper arm
    elements.append(create_rectangle(posi=[posi[0],posi[1]-arms_offset],dims=[boxwidth,boxlength],anchor='center')) # lower arm
    elements.append(create_rectangle(posi=[posi[0]-arms_offset,posi[1]],dims=[boxlength,boxwidth],anchor='center')) # left arm
    elements.append(create_rectangle(posi=[posi[0]+arms_offset,posi[1]],dims=[boxlength,boxwidth],anchor='center')) # right arm

    marker = merge_polygons(elements)

    return marker


def create_marker_matrix(posi_marker1,width_marker,repetitions,offsets):
    """ Creates an array of markers
    
    @param list center_marker1: list of floats that gives the position (x and y) of the marker in the lower left corner of the array

    @param float width_marker: size of the marker

    @param list repetitions: list of ints with the number of repetitions in x and y direction

    @param list offsets: list of floats that gives the offset between markers in x and y direction

    @return Region: region that contains the markers
    """

    markers = []
    center = [0,0] # initialize center
    for i in range(repetitions[0]):
        center[0] = posi_marker1[0] + i*offsets[0]
        for j in range(repetitions[1]):
            center[1] = posi_marker1[1] + j*offsets[1]
            marker = create_marker(center,width_marker)
            markers.append(marker)
    matrix = merge_regions(markers)
    
    return matrix


def create_numbers_for_marker(posi,gridsize,crosswidth,numbers): #posi
    """Creates 4 numbers in a square.

    If set up correctly, numbers should be in quadrants of the corresponding marker/cross.

    @param list center: x and y coordinate of the crosses center

    @param float gridsize: size of the used grid (layout.dbu)

    @param float crosswidth: width of the corresponding cross

    @param list numbers: numbers as string that should be placed in the quadrants.
        The order is a follows: [top left, top right, lower left, lower right]
    """
    textheight = 0.3*crosswidth
    elements = []
    elements.append(create_text([posi[0]-crosswidth/4,posi[1]+crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[0],
                                anchor = 'c')
                    ) # top left
    elements.append(create_text([posi[0]+crosswidth/4,posi[1]+crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[1],
                                anchor = 'c')
                    ) # top right
    elements.append(create_text([posi[0]-crosswidth/4,posi[1]-crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[2],
                                anchor = 'c')
                    ) # bottom left
    elements.append(create_text([posi[0]+crosswidth/4,posi[1]-crosswidth/4],
                                gridsize,
                                textheight,
                                numbers[3],
                                anchor = 'c')
                    ) # bottom right

    numbers = merge_regions(elements)

    return numbers


def create_number_matrix(posi_marker1,gridsize,width_marker,repetitions,offsets): #posi
    """ Creates an array of numbers.
    
    @param list center_marker1: list of floats that gives the position (x and y) of the corresponding marker in the lower left corner of the array

    @param float width_marker: size of the corresponding marker

    @param list repetitions: list of ints with the number of repetitions in x and y direction

    @param list offsets: list of floats that gives the offset between corresponding markers in x and y direction

    @return Region: region that contains the numbers
    """
    elements = []
    center = [0,0] # initialize center
    for i in range(repetitions[0]):
        center[0] = posi_marker1[0] + i*offsets[0]
        num_x = f'{i:02}'
        for j in range(repetitions[1]):
            center[1] = posi_marker1[1] + j*offsets[1]
            num_y = f'{j:02}'
            numbers = [num_y[:-1],num_y[-1],num_x[:-1],num_x[-1]] # takes last number in right quadrants, rest (usually only 1) in left quadrants
            nums = create_numbers_for_marker(center,gridsize,width_marker,numbers)
            elements.append(nums)
    matrix = merge_regions(elements)

    return matrix


def create_wg_tapered(posi=[0,0],width=600,length=10000,taper_length_tip_top=0,taper_length_tip_bottom=0):
    '''Creates a tapered waveguide with triangular corrugations on the side.
    
    @param list of int posi: center coordinates of the waveguide (without taper) as [x,y].

    @param int width: width of the waveguide.

    @param int length: length of the waveguide.

    @param int taper_length_tip_top: distance over which the width tapering on the top takes place.

    @param int taper_length_tip_bottom: distance over which the width tapering on the bottom takes place.

    @return wg: Region object of the waveguide.

    TODO: make possibility to taper depth of triangles.
    '''

    left = posi[0]-width/2
    right = posi[0]+width/2
    top = posi[1]+length/2
    bottom = posi[1]-length/2

    points = []

    # lower left corner
    x = left
    y = bottom
    points.append([x,y])

    # upper left corner
    x = left
    y = top
    points.append([x,y])

    # tip/taper top
    if taper_length_tip_top != 0:
        x = posi[0]
        y = top+taper_length_tip_top
        points.append([x,y])

    # upper right corner
    x = right
    y = top
    points.append([x,y])

    # lower right corner
    x = right
    y = bottom
    points.append([x,y])

    # tip/taper bottom
    if taper_length_tip_bottom != 0:
        x = posi[0]
        y = bottom-taper_length_tip_bottom
        points.append([x,y])

    # last point same as first point
    [x,y] = points[0]
    points.append([x,y])

    poly = create_polygon(points)

    # polygon needs to be turned into region
    wg = pya.Region(poly)

    return wg


def create_wg_corrugated_triangles(posi=[0,0],inner_width=200,outer_width=600,period_length=216,corrs_stop=1.0,n_corrs=10,width=600,length=10000,taper_length_tip=20000):
    '''Creates a tapered waveguide with triangular corrugations on the side.
    
    @param list of int posi: center coordinates of the waveguide (without taper) as [x,y].

    @param int inner_width: inner width of the waveguide (triangle base to triangle base).

    @param int outer_width: outer width of the waveguide (triangle tip to triangle tip).

    @param int period_length: period of the corrugations (triangle tip to triangle tip).
    
    @param int width: width of the waveguide.

    @param int length: length of the waveguide.

    @param float corrs_stop: fraction of the beam length when the corrugations should stop. (e.g. 1.0: after 100% of the beam length from the bottom, i.e. at the top)

    @param int n_corrs: number of corrugations. Plus one weird additional one to go from wg width to corrugation width.

    @param int taper_length_tip: distance over which the width tapering on the bottom takes place.

    @return wg: Region object of the waveguide.

    TODO: make possibility to taper depth of triangles.
    '''
    x_posis_left = [posi[0]-outer_width/2,posi[0]-inner_width/2]
    x_posis_right = [posi[0]+outer_width/2,posi[0]+inner_width/2]
    stop = posi[1]+length*(corrs_stop-0.5)
    start = stop - (n_corrs+0.5)*period_length
    y_posis_left = np.linspace(start=start,stop=stop,num=n_corrs*2+2)
    y_posis_right = y_posis_left[::-1] #reverse order

    points = []

    # lower left corner
    x = posi[0]-width/2
    y = posi[1]-length/2
    points.append([x,y])

    # start triangles/end wg on left side
    x = posi[0]-width/2
    y = y_posis_left[0] - (y_posis_left[1]-y_posis_left[0])
    points.append([x,y])

    # corrugations left side
    i = 0 # start on the inside
    for y in y_posis_left:
        x = x_posis_left[i]
        points.append([x,y])
        i = (i+1)%2
    
    # corrugations right side
    i = 1 # start on the inside
    for y in y_posis_right:
        x = x_posis_right[i]
        points.append([x,y])
        i = (i+1)%2

    # stop triangles right side
    x = posi[0]+width/2
    y = y_posis_right[-1] - (y_posis_right[-2] - y_posis_right[-1])
    points.append([x,y])

    # lower right corner
    x = posi[0]+width/2
    y = posi[1]-length/2
    points.append([x,y])

    # tip/taper bottom
    if taper_length_tip != 0:
        x = posi[0]
        y = posi[1]-length/2-taper_length_tip
        points.append([x,y])

    # last point same as first point
    [x,y] = points[0]
    points.append([x,y])

    poly = create_polygon(points)

    # polygon needs to be turned into region
    wg = pya.Region(poly)

    return wg


def create_frame(outer_lower_left=(0,0), inner_lower_left=(100000,100000), outer_dim=(400000,400000), inner_dim=(200000,200000)):
    '''
    '''
    x_oll = outer_lower_left[0]
    y_oll = outer_lower_left[1]
    x_olr = x_oll + outer_dim[0]
    # y_olr = y_oll
    # x_oul = x_oll
    # y_oul = y_oll + outer_dim[1]
    x_our = x_oll + outer_dim[0]
    y_our = y_oll + outer_dim[1]

    x_ill = inner_lower_left[0]
    y_ill = inner_lower_left[1]
    x_ilr = x_ill + inner_dim[0]
    y_ilr = y_ill
    # x_iul = x_ill
    y_iul = y_ill + inner_dim[1]
    # x_iur = x_ill + inner_dim[0]
    y_iur = y_ill + inner_dim[1]

    struct = []
    
    struct.append(create_rectangle([x_oll,y_oll,x_olr,y_ilr]))
    struct.append(create_rectangle([x_ilr,y_ilr,x_olr,y_iur]))
    struct.append(create_rectangle([x_oll,y_ill,x_ill,y_iur]))
    struct.append(create_rectangle([x_oll,y_iul,x_our,y_our]))

    struct = merge_regions(struct)
    return struct


def create_support(posi=(0,0), support_width=500, support_length=20000, base_width=5000, base_length=20000, n_beams=2):
    """Creates a support structure with center at posi.
    
    Support structure consists of frame with wide beams on the left and right.

    @param list posi: center coordinates of the waveguide (without taper) as [x,y].

    @param int support_width: width of the horizontal support beam

    @param int support_length: length of the horizontal support beam

    @param int base_width: width of the base to the left and right

    @param int base_length: height of the base to the left and right

    @param int n_beams: number of support beams
    """
    center_right_base = (support_length+base_width)/2
    center_left_base = -center_right_base
    center_upper_support = (base_length-support_width)/2
    center_lower_support = -center_upper_support

    struct = []
    struct.append(create_rectangle(posi=[center_left_base,0], dims=[base_width,base_length], anchor='center')) #originally lines below
    struct.append(create_rectangle(posi=[center_right_base,0], dims=[base_width,base_length], anchor='center'))
    if n_beams >= 2:
        n_beams=int(n_beams)
    else:
        n_beams = 2
    for i in range(n_beams):
        y = center_lower_support + i/(n_beams-1)*(center_upper_support-center_lower_support)
        struct.append(create_rectangle(posi=[0,y],dims=[support_length,support_width], anchor='center'))

    poly = merge_polygons(struct)

    boundingbox = poly.bbox()
    x = -(boundingbox.left+boundingbox.right)/2 + posi[0]
    y = -(boundingbox.bottom+boundingbox.top)/2 + posi[1]
    poly = move_region(poly,(x,y))

    return poly


def create_bezier_support(posi=[0,0], length=2000, width_inner=600, width_outer=800, npoints=100):
    y2 = posi[1]
    y0 = y2 - length/2
    y1 = (y0+y2)/2
    y4 = y2 + length/2
    y3 = (y2+y4)/2
    x0 = posi[0] + width_outer/2
    x1 = posi[0] + width_inner/2
    x2 = posi[0] - width_inner/2
    x3 = posi[0] - width_outer/2
    # control_points_up = [[x0,y1], [x1,y1], [x1,y0], [x2,y0], [x2,y0], [x3,y0], [x3,y1], [x4,y1]]
    control_points_up = [[x1,y0], [x1,y1], [x0,y1], [x0,y2], [x0,y3], [x1,y3], [x1,y4]]
    # control_points_down =[[x4,y2], [x3,y2], [x3,y3], [x2,y3], [x1,y3], [x1,y2], [x0,y2]]
    control_points_down =[[x2,y4], [x2,y3], [x3,y3], [x3,y2], [x3,y1], [x2,y1], [x2,y0]]

    upper = mbz.BezierSegment(control_points=control_points_up)
    lower = mbz.BezierSegment(control_points=control_points_down)

    tt = np.linspace(0,1,int(npoints/2))

    curve1 = upper.point_at_t(tt)
    curve1 = np.array(curve1)
    curve2 = lower.point_at_t(tt)
    curve2 = np.array(curve2)
    curve = np.vstack([curve1,curve2])
    curve = np.vstack([curve,curve[0]])
    
    support = create_polygon(curve)

    return support


def create_circle_supports(x_coordinates=[-5000,5000],y_coordinates=[-5000,5000],diameter=700):
    struct = []
    for x in x_coordinates:
        for y in y_coordinates:
            struct.append(create_circle((x,y),diameter/2,n_points=40))

    poly = merge_polygons(struct)

    return poly


def circle(center,radius,n_points):
    """returns points that form a circle
    
    @param int n_points: number of points that make up the circle
    
    @param float radius: radius of the circle
    
    @param list center: center coordinates of the circle
    """
    pts = []
    for i in range(0,n_points+1): # first and last point are identical
        angle = 2*np.pi*i/n_points
        x = center[0] + radius*np.cos(angle)
        y = center[1] + radius*np.sin(angle)
        pts.append((x,y))

    return pts



def partial_circle(center, radius, n_points, start_angle=0, end_angle=np.pi, add_center=True):
    """
    Returns points that form a partial circle (arc), with an optional center point.
    
    @param list center: center coordinates of the circle
    @param float radius: radius of the circle
    @param int n_points: number of points that make up the arc
    @param float start_angle: starting angle of the arc in radians (default 0)
    @param float end_angle: ending angle of the arc in radians (default π for a half-circle)
    @param bool add_center: whether to add the center point (default True, only for partial circles)
    """
    pts = []
    
    # Add center point if it's not a full circle
    if add_center and abs(end_angle - start_angle) < 2 * np.pi:
        pts.append(tuple(center))
    
    for i in range(n_points+1):  # first and last point are included
        angle = start_angle + (end_angle - start_angle) * i / n_points
        x = center[0] + radius * np.cos(angle)
        y = center[1] + radius * np.sin(angle)
        pts.append((x, y))

    return pts


def create_circle(center,radius,n_points):
    """Creates a circle object.
    
    @param int n_points: number of points that make up the circle
    
    @param float radius: radius of the circle
    
    @param list center: center coordinates of the circle

    @return poly: Circle as polygon object
    """
    structure = []
    for i in circle(center,radius,n_points):
        structure.append(pya.Point.new(i[0],i[1]))

    poly = pya.Polygon.new(structure)
    return poly


def create_partial_circle(center, radius, n_points, start_angle=0, end_angle=np.pi):
    """Creates a partial circle object.
    
    @param int n_points: number of points that make up the circle
    
    @param float radius: radius of the circle
    
    @param list center: center coordinates of the circle

    @param float start_angle: starting angle of the arc in radians (default 0)

    @param float end_angle: ending angle of the arc in radians (default π for a half-circle)

    @return poly: Circle as polygon object
    """
    structure = []
    for i in partial_circle(center,radius,n_points,start_angle,end_angle):
        structure.append(pya.Point.new(i[0],i[1]))

    poly = pya.Polygon.new(structure)
    return poly


def create_partial_circle_arc(center, radius, width, n_points, start_angle=0, end_angle=np.pi):
    """Creates a partial circle object.
    
    @param int n_points: number of points that make up the circle
    
    @param float radius: radius of the middleline of the arc

    @param float width: width of the arc
    
    @param list center: center coordinates of the circle

    @param float start_angle: starting angle of the arc in radians (default 0)

    @param float end_angle: ending angle of the arc in radians (default π for a half-circle)

    @return poly: Circlearc as polygon object
    """
    structure1 = []
    structure2 = []

    r1 = radius + width/2
    r2 = radius - width/2

    for i in partial_circle(center,r1,n_points,start_angle,end_angle):
        structure1.append(pya.Point.new(i[0],i[1]))
    poly1 = pya.Polygon.new(structure1)
    region1 = merge_regions(poly1)
    for i in partial_circle(center,r2,n_points,start_angle,end_angle):
        structure2.append(pya.Point.new(i[0],i[1]))
    poly2 = pya.Polygon.new(structure2)
    region2 = merge_regions(poly2)

    region = region1 - region2

    return region

def create_electrode_pair(posi=[0,0], electrode_width=10000, electrode_length=100000, pad_width=200000, pad_length=200000, electrode_separation=10000):
    # anchor is lower center, aka. south
    struct = []
    x2 = posi[0]-electrode_separation/2
    y0 = posi[1] + pad_length+electrode_length
    y1 = posi[1] + electrode_length
    
    struct.append(create_rectangle(corners=[x2-pad_width,y1,x2,y0]))
    struct.append(create_rectangle(corners=[x2-electrode_width,posi[1],x2,y1]))
    x3 = x2+electrode_separation
    struct.append(create_rectangle(corners=[x3,y1,x3+pad_width,y0]))
    struct.append(create_rectangle(corners=[x3,posi[1],x3+electrode_width,y1]))

    struct = merge_regions(struct)
    return struct


def create_bezier_curve(control_points,npoints=20):
    """control points should look like this: 
        [[x1,y1],[x2,y2],[x3,y3],...]
        @param int npoints: number of points that make up the finished curve.
        """
    bez = mbz.BezierSegment(control_points=control_points)

    tt = np.linspace(0,1,int(npoints))

    curve = bez.point_at_t(tt)
    curve = np.array(curve)

    return curve


def create_curved_eclectrode_pair(posi=[0,0], 
                                  thin_electrode_width=1000, 
                                  thick_electrode_width=10000, 
                                  thin_electrode_length=100000, 
                                  thick_electrode_length=100000,
                                  thin_electrode_separation=1000,
                                  thick_electrode_separation=10000,
                                  pad_width=200000,
                                  pad_length=200000):
    # anchor is lower center, aka. south
    x4 = posi[0]-thin_electrode_separation/2
    x3 = x4-thin_electrode_width
    x2 = posi[0]-thick_electrode_separation/2
    x1 = x2-thick_electrode_width
    x0 = x2-pad_width

    x5 = posi[0]+thin_electrode_separation/2
    x6 = x5+thin_electrode_width
    x7 = posi[0]+thick_electrode_separation/2
    x8 = x7+thick_electrode_width
    x9 = x7+pad_width

    y3 = posi[1]
    y2 = y3+thin_electrode_length
    y1=y2+thick_electrode_length
    y0 = y1+pad_length

    struct = []
    #left
    struct.append(create_rectangle(corners=[x3,y3,x4,y2])) # thin electrode
    # curved electrode
    c1 = create_bezier_curve([[x4,y2],[x4,(y1+y2)/2],[(x2+x4)/2,(y1+y2)/2],[x2,(y1+y2)/2],[x2,y1]])
    c2 = create_bezier_curve([[x1,y1],[x1,(y1+y2)/2],[(x1+x3)/2,(y1+y2)/2],[x3,(y1+y2)/2],[x3,y2]])
    c = np.vstack([c1,c2])
    c= np.vstack([c,c[0]])
    struct.append(create_polygon(c))
    struct.append(create_rectangle(corners=[x0,y1,x2,y0])) # pad
    #right
    struct.append(create_rectangle(corners=[x6,y3,x5,y2])) # thin electrode
    # curved electrode
    c1 = create_bezier_curve([[x5,y2],[x5,(y1+y2)/2],[(x7+x5)/2,(y1+y2)/2],[x7,(y1+y2)/2],[x7,y1]])
    c2 = create_bezier_curve([[x8,y1],[x8,(y1+y2)/2],[(x8+x6)/2,(y1+y2)/2],[x6,(y1+y2)/2],[x6,y2]])
    c = np.vstack([c1,c2])
    c= np.vstack([c,c[0]])
    struct.append(create_polygon(c))
    struct.append(create_rectangle(corners=[x9,y1,x7,y0])) # pad

    struct = merge_regions(struct)
    return struct


def create_curved_eclectrode_pair_2(posi=[0,0], 
                                  thin_electrode_width=1000, 
                                  thick_electrode_width=10000, 
                                  thin_electrode_length=100000, 
                                  thick_electrode_length=100000,
                                  thin_electrode_separation=1000,
                                  thick_electrode_separation=10000,
                                  pad_width=200000,
                                  pad_length=200000):
    # anchor is lower center, aka. south
    x4 = posi[0]-thin_electrode_separation/2
    x3 = x4-thin_electrode_width
    x2 = posi[0]-thick_electrode_separation/2
    x1 = x2-thick_electrode_width
    x0 = x2-pad_width

    x5 = posi[0]+thin_electrode_separation/2
    x6 = x5+thin_electrode_width
    x7 = posi[0]+thick_electrode_separation/2
    x8 = x7+thick_electrode_width
    x9 = x7+pad_width
    x10 = x8-pad_width

    y3 = posi[1]
    y2 = y3+thin_electrode_length
    y1=y2+thick_electrode_length
    y0 = y1+pad_length

    struct = []
    #left
    struct.append(create_rectangle(corners=[x3,y3,x4,y2])) # thin electrode
    # curved electrode
    c1 = create_bezier_curve([[x4,y2],[x4,(y1+y2)/2],[(x2+x4)/2,(y1+y2)/2],[x2,(y1+y2)/2],[x2,y1]])
    c2 = create_bezier_curve([[x1,y1],[x1,(y1+y2)/2],[(x1+x3)/2,(y1+y2)/2],[x3,(y1+y2)/2],[x3,y2]])
    c = np.vstack([c1,c2])
    c= np.vstack([c,c[0]])
    struct.append(create_polygon(c))
    struct.append(create_rectangle(corners=[x0,y1,x2,y0])) # pad
    #right
    struct.append(create_rectangle(corners=[x6,y3,x5,y2])) # thin electrode
    # curved electrode
    c1 = create_bezier_curve([[x5,y2],[x5,(y1+y2)/2],[(x7+x5)/2,(y1+y2)/2],[x7,(y1+y2)/2],[x7,y1]])
    c2 = create_bezier_curve([[x8,y1],[x8,(y1+y2)/2],[(x8+x6)/2,(y1+y2)/2],[x6,(y1+y2)/2],[x6,y2]])
    c = np.vstack([c1,c2])
    c= np.vstack([c,c[0]])
    struct.append(create_polygon(c))
    y01 = y0+thick_electrode_separation
    y02 = y01 + pad_length
    struct.append(create_rectangle(corners=[x8,y1,x7,y01]))# extension right wire
    struct.append(create_rectangle(corners=[x10,y01,x8,y02])) # pad

    struct = merge_regions(struct)
    return struct


def create_curved_eclectrode_pair_3(posi=[0,0], 
                                  thin_electrode_width=1000, 
                                  thick_electrode_width=100000, 
                                  thin_electrode_length=100000, 
                                  thick_electrode_length=100000,
                                  thin_electrode_separation=3000,
                                  pad_separation=100000,
                                  pad_width=200000,
                                  pad_length=200000):
    # anchor is lower center between electrodes, aka. south
    x0 = posi[0]-thin_electrode_separation/2-thin_electrode_width
    x1 = posi[0]-thin_electrode_separation/2
    x2 = x0 + thick_electrode_width
    x3 = x0 + pad_width

    x4 = posi[0]+thin_electrode_separation/2
    x5 = x4 + thin_electrode_width
    x6 = x3 + pad_separation
    x8 = x6 + pad_width
    x7 = x8 - thick_electrode_width

    y3 = posi[1]
    y2 = y3+thin_electrode_length
    y2p = y2 + thick_electrode_length*0.1
    y1 = y2+thick_electrode_length
    y0 = y1+pad_length

    struct = []
    # left
    struct.append(create_rectangle(corners=[x0,y3,x1,y2p])) # thin electrode
    # curved electrode
    c1 = np.array([[x0,y2p],[x0,y1]])
    c2 = create_bezier_curve([[x2,y1],[x2,(y1+y2p)/2],[(x1+x2)/2,(y1+y2p)/2],[x1,(y1+y2p)/2],[x1,y2p]])
    c = np.vstack([c1,c2])
    c= np.vstack([c,c[0]])
    struct.append(create_polygon(c))
    struct.append(create_rectangle(corners=[x0,y1,x3,y0])) # pad
    # right
    struct.append(create_rectangle(corners=[x4,y3,x5,y2])) # thin electrode
    # curved electrode
    c1 = create_bezier_curve([[x4,y2],[x4,(y1+y2)/2],[(x7+x4)/2,(y1+y2)/2],[x7,(y1+y2)/2],[x7,y1]])
    c2 = create_bezier_curve([[x8,y1],[x8,(y1+y2)/2],[(x8+x5)/2,(y1+y2)/2],[x5,(y1+y2)/2],[x5,y2]])
    c = np.vstack([c1,c2])
    c= np.vstack([c,c[0]])
    struct.append(create_polygon(c))
    struct.append(create_rectangle(corners=[x6,y1,x8,y0])) # pad

    struct = merge_regions(struct)
    return struct


def create_interleafed_comb(posi=[0,0], electrode_width=10000, electrode_length=50000, electrode_separation=30000, n_electrodes=3, comb_separation=70000, comb_length=500000, comb_width=50000, pad_width=100000, pad_length=100000):
    # anchor = west
    y1 = posi[1] + comb_separation/2
    y0 = y1 + comb_width
    y2 = y1 - electrode_length
    
    y4 = posi[1] - comb_separation/2
    y5 = y4 - comb_width
    y3 = y4 + electrode_length

    struct = []
    struct.append(create_rectangle(corners=[posi[0],y1,posi[0]+comb_length,y0]))
    for i in range(n_electrodes):
        x = posi[0] + 2*i*(electrode_separation + electrode_width) 
        struct.append(create_rectangle(corners=[x,y2,x+electrode_width,y1]))
    struct.append(create_rectangle(corners=[posi[0],y5,posi[0]+comb_length,y4]))
    for i in range(n_electrodes):
        x = posi[0] + (2*i+1)*(electrode_separation + electrode_width) 
        struct.append(create_rectangle(corners=[x,y4,x+electrode_width,y3]))
    
    x1 = posi[0] + comb_length
    struct.append(create_rectangle(corners=[x1,y1,x1+pad_length,y1+pad_width]))
    struct.append(create_rectangle(corners=[x1,y4,x1+pad_length,y4-pad_width]))

    struct = merge_regions(struct)

    return struct


def create_pillar_array(posi=[0,0], radius=500, separation=5000, nx=10, ny = 10, skip_corner_pillars=0, n_points_circle=20):
    # anchor = center
    ll = [posi[0]-(nx-1)/2*separation,posi[1]-(ny-1)/2*separation]
    ur = [ll[0]+(nx-1)*separation,ll[1]+(ny-1)*separation]
    xx = np.linspace(ll[0],ur[0],nx,dtype=int)
    yy = np.linspace(ll[1],ur[1],ny,dtype=int)
    struct = []
    for x in xx:
        for y in yy:
            # check if we are in corner
            bool_x1 = x < ll[0]+separation*skip_corner_pillars # left side
            bool_x2 = x > ur[0]-separation*skip_corner_pillars # right side
            bool_x = bool_x1 or bool_x2
            bool_y1 = y < ll[1]+separation*skip_corner_pillars # bottom
            bool_y2 = y > ur[1]-separation*skip_corner_pillars # top
            bool_y = bool_y1 or bool_y2
            bool_xy = bool_x and bool_y
            if not bool_xy:
                struct.append(create_circle((x,y),radius,n_points_circle))
    struct = merge_regions(struct)
    return struct


def create_dummy_array(posi=(0,0),wg_width=500,wg_length=10000, n_wgs=4, perc_widths=None):
    if perc_widths != None:
        n_wgs = len(perc_widths)

    struct = []
    
    separation= 3*wg_width
    x_posis = np.linspace(-n_wgs/2*separation,n_wgs/2*separation,n_wgs)
    
    for i in range(n_wgs):
        x = x_posis[i]
        if perc_widths != None:
            perc = perc_widths[i]
            width = int(perc*wg_width)
        else:
            width = int((i+1)/n_wgs*wg_width)
        struct.append(create_rectangle(posi=[x,0],dims=[width,wg_length],anchor='center'))

    struct = merge_regions(struct)
    
    x = posi[0]
    y = posi[1]
    struct = move_region(struct,(x,y))

    return struct


def create_wg_with_support(posi, width, length, taper_top, taper_bottom, n_support, width_support):
    [x,y] = posi
    wg1 = create_wg_tapered(posi=[x,y],
                            width=width, 
                            length=length, 
                            taper_length_tip_bottom=taper_bottom, 
                            taper_length_tip_top=taper_top)
    y_support = np.linspace(y-length*0.45,y+length*0.45,n_support)
    for y in y_support:
        support = create_bezier_support(posi=[x,y],width_inner=width,width_outer=width_support)
        wg1 = merge_regions([wg1,support])
    return wg1


def create_waveguide_with_return_loop(posi,width,radius,length,n_points,taper_length):
    """

    @ param list posi: center of the center line of the return loop on top.
    """
    [x0,y0] = posi
    x1 = x0-radius-width/2
    x2 = x0-radius
    x3 = x0-radius+width/2
    x4 = x0+radius-width/2
    x5 = x0+radius
    x6 = x0+radius+width/2
    y1 = y0-length
    y2=y1-taper_length
    arc = create_partial_circle_arc(center=posi,radius=radius,width=width,n_points=n_points,start_angle=0,end_angle=np.pi)
    waveguide1 = [[x1,y0],[x1,y1],[x2,y2],[x3,y1],[x3,y0],[x1,y0]]
    waveguide1 = create_polygon(waveguide1)
    waveguide2 = [[x4,y0],[x4,y1],[x5,y2],[x6,y1],[x6,y0],[x4,y0]]
    waveguide2 = create_polygon(waveguide2)
    waveguide = [waveguide1,arc,waveguide2]
    waveguide = merge_regions(waveguide)
    
    return waveguide


def semicircle_with_box(posi, radius, boxheight, n_points):
    """
    Returns points that form a semicircle with a box underneath.
    
    @param list posi: coordinates of the south end of the structure
    @param float radius: radius of the circle
    @param float boxheight: height of the box beneath the semicircle
    @param int n_points: number of points that make up the arc
    """

    start_angle=0
    end_angle=np.pi

    pts = []
    [x0,y0]=posi
    x = posi[0]+radius
    y=posi[1]
    pts.append((x,y))
    
    x0 = posi[0]
    y0 = posi[1]+boxheight
    for i in range(n_points+1):  # first and last point are included
        angle = start_angle + (end_angle - start_angle) * i / n_points
        x = x0 + radius * np.cos(angle)
        y = y0 + radius * np.sin(angle)
        pts.append((x, y))

    [x0,y0]=posi
    x = posi[0]-radius
    y=posi[1]
    pts.append((x,y))

    return pts


def create_protective_semicircle(posi, radius_inner, radius_outer, buffer_front, n_points):
    """Creates a substracted circle object. 

    Takes two circles with aligned midpoints. Substracts them to get an acrish shape.
    
    @param int n_points: number of points that make up the circle
    
    @param float radius: radius of the middleline of the arc

    @param float width: width of the arc at the bottom
    
    @param list posi: s anchor of the structure

    @param float start_angle: starting angle of the arc in radians (default 0)

    @param float end_angle: ending angle of the arc in radians (default π for a half-circle)

    @return poly: Circlearc as polygon object
    """
    structure1 = []
    structure2 = []

    r1 = radius_outer
    r2 = radius_inner

    for i in semicircle_with_box(posi,r1,boxheight=buffer_front,n_points=n_points):
        structure1.append(pya.Point.new(i[0],i[1]))
    poly1 = pya.Polygon.new(structure1)
    region1 = merge_regions(poly1)
    for i in semicircle_with_box(posi,r2,boxheight=0,n_points=n_points):
        structure2.append(pya.Point.new(i[0],i[1]))
    poly2 = pya.Polygon.new(structure2)
    region2 = merge_regions(poly2)

    region = region1 - region2

    return region



def create_protection_for_waveguide_with_return_loop(posi,width,radius,length,width_hammerhead,buffer_front):
    semicirc = create_protective_semicircle(posi=[posi[0],posi[1]-buffer_front], radius_inner=radius-width/2, radius_outer=radius+width/2, buffer_front=buffer_front, n_points=100)
    [x0,y0] = posi
    y0 = y0 - length
    box1 = create_rectangle(posi=[x0-radius,y0], dims=[width,length], anchor='s')
    box2 = create_rectangle(posi=[x0-radius,y0], dims=[width_hammerhead,width], anchor='s')
    box3 = create_rectangle(posi=[x0+radius,y0], dims=[width,length], anchor='s')
    box4 = create_rectangle(posi=[x0+radius,y0], dims=[width_hammerhead,width], anchor='s')

    struct = merge_regions([semicirc,box1,box2,box3,box4])

    return struct

In [28]:
################################################################################
## create new layout
layout = pya.Layout()
layout.dbu = 1e-3 # data base unit in um --> each number we put is now in nm

top = layout.create_cell("TOP")
l0 = layout.layer(0,0)
l01 = layout.layer(0,1)
l1 = layout.layer(1,0)
l2 = layout.layer(2,0)
le = layout.layer(4,0)
lwf = layout.layer(254,0)
la = layout.layer(255,0)

################################################################################
x_shift,y_shift = 120000,95000

structs = []
protects = []
x0 = 0
y0 = 0


width = 350
length = 50000
taper = 6000 # 10um * 350nm/600nm
sep = 10000

facs = [1.5, 1.7, 1.9]


for i in range(len(facs)):
    fac = facs[i]
    _y = y0 + i*y_shift
    _x = x0
    for j in range(2):
        dummies = create_dummy_array(posi=[_x-10*width-sep,_y],wg_width=width,wg_length=length,perc_widths=[0.25,0.5,0.75,0.9,0.95,1,1.05])
        structs.append(dummies)
        for k in range(10):
            wg = create_wg_with_support(posi=[_x,_y], width=width, length=length, taper_top=0, taper_bottom=taper, n_support=2, width_support=width*fac)
            structs.append(wg)
            _x = _x + sep
        _x = x0 + (j+1)*x_shift

################################################################################
## create markers
width_marker = 10000
posi_marker1 = [-15000,-60000]
offsets = [x_shift,y_shift]

repetitions = [3,4]
marker_matrix = create_marker_matrix(posi_marker1=posi_marker1,
                                        width_marker=width_marker,
                                        repetitions=repetitions, 
                                        offsets=offsets)
number_matrix = create_number_matrix(posi_marker1=posi_marker1,
                                        gridsize=layout.dbu,
                                        width_marker=width_marker,
                                        repetitions=repetitions,
                                        offsets=offsets)


################################################################################
## frame
wall_thickness = 80000
distance_to_marker = 20000
ill = [posi_marker1[0]-width_marker/2-distance_to_marker, posi_marker1[1]-width_marker/2-distance_to_marker]
oll = [ill[0]-wall_thickness,ill[1]-wall_thickness]
inner_dims = (offsets[0]*(repetitions[0]-1)+width_marker+2*distance_to_marker,offsets[1]*(repetitions[1]-1)+width_marker+2*distance_to_marker)
outer_dims = [inner_dims[0]+2*wall_thickness,inner_dims[1]+2*wall_thickness]
frame = create_frame(oll,ill,outer_dims,inner_dims)

## writefield & annotations
wfs = []
for i in range(2):
    for j in range(2):
        x = oll[0]+i*500000
        y = oll[1]+j*500000
        wf=create_rectangle(posi=[x,y],dims=[500000,500000],anchor='sw')
        wfs.append(wf)

annots = []
annotation_str='%d x %d um'%(outer_dims[0]*1e-3,outer_dims[1]*1e-3)

annot = create_text(posi=oll,gridsize=layout.dbu,height=50000,text=annotation_str,anchor='sw')
annots.append(annot)

################################################################################
## place shapes
top.shapes(l0).insert(marker_matrix)
top.shapes(l01).insert(number_matrix)
# top.shapes(l02).insert(marker_matrix_bm)
for struct in structs:
    top.shapes(l1).insert(struct)
for protect in protects:
    top.shapes(l2).insert(protect)
top.shapes(l2).insert(frame)
for annot in annots:
    top.shapes(la).insert(annot)
for wf in wfs:
    top.shapes(lwf).insert(wf)

## save in file
layout.write('ttt.gds')